In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf 
import cv2
import matplotlib.pyplot as plt 
from keras import Sequential
from keras.layers import Conv2D , Dropout, MaxPooling2D  ,Dense 
from keras.activations import softmax
from IPython.display import Image
from sklearn.model_selection import train_test_split
from keras import Input
import zipfile
from keras.utils.np_utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten

In [ ]:
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r") as zip_ref:
    zip_ref.extractall(".")
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip","r") as zip_ref:
    zip_ref.extractall(".")

# Extracting Labels

In [ ]:
y = [0 if i.split('.')[0]=='cat' else 1 for i in os.listdir("./train")]

In [ ]:
print(y[0:5],os.listdir("./train")[0:5])

In [ ]:
np.unique(y , return_counts=True)

In [ ]:
y = to_categorical(y,num_classes=2)

# Exploring data

In [ ]:
plt.figure(figsize=(10,10))
for i in range(1,10):
    ax=plt.subplot(3, 3, i)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    img=cv2.imread("./train/"+os.listdir("./train")[i])
    ax.set_title(y[i])
    plt.imshow(img)
print(img.shape)

# Build training set

In [ ]:
path="./train"
X=[]
for i in os.listdir(path):
    img_path = os.path.join(path, i)
    img = cv2.imread(img_path,0)
    img = cv2.resize(img,(80,80))
    X.append(img)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(1,10):
    ax=plt.subplot(3, 3, i)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    #img=cv2.imread("./train/"+os.listdir("./train")[i])
    img = X[i]
    ax.set_title(y[i])
    plt.imshow(img , cmap='gray')
print(img.shape)

In [ ]:
X= np.array(X)

In [ ]:
X_dim= np.expand_dims(X , axis=3)

In [ ]:
data_gen = ImageDataGenerator(rescale = 1./255 , validation_split=0.25)

In [ ]:
data_gen.fit(X_dim)

# Model Arc

In [ ]:
def model_gen():
    model=tf.keras.Sequential()
    ################################### FEATURE EXTRACTOR 
    model.add(Conv2D(32,3,activation='relu',input_shape=(80, 80, 1)))
    model.add(Conv2D(32,4,activation='relu'))
    model.add(MaxPooling2D(2))
    model.add(Dropout(0.3))
    model.add(Conv2D(64,3,activation='relu'))
    model.add(Conv2D(64,4,activation='relu'))
    model.add(MaxPooling2D(2))
    model.add(Dropout(0.3))
    model.add(Conv2D(84,3,activation='relu'))
    model.add(Dropout(0.3))
    #################################### Classifier 
    model.add(Flatten())
    model.add(Dense(32,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(2, activation='softmax')) 
    model.compile(optimizer='Adamax',loss=['CategoricalCrossentropy'],
                  metrics=['categorical_accuracy'])
    tf.keras.utils.plot_model(model, 'model.png',show_shapes=True)
    
    return model

model=model_gen()
Image('model.png')

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss',restore_best_weights=True),
    tf.keras.callbacks.LearningRateScheduler(scheduler)]

In [ ]:
X_dim.shape

In [ ]:
history=model.fit(data_gen.flow(X_dim, y, batch_size=64,subset='training'),
          validation_data=data_gen.flow(X_dim, y,batch_size=8, subset='validation'),epochs=25,callbacks=callbacks, shuffle=True)

# Learning Curve

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_dim,y,train_size=0.8, test_size=0.2,stratify=y, random_state=0)

In [ ]:
model2=model_gen()

In [ ]:
history=model2.fit(X_train, y_train, epochs=25,batch_size=64,
          validation_data=(X_valid,y_valid),callbacks=callbacks, shuffle=True)

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(X_valid,y_valid)

In [ ]:
model2.evaluate(X_valid,y_valid)

In [ ]:
from sklearn.metrics import confusion_matrix , f1_score , classification_report

In [ ]:
y_pred = model2.predict(X_valid)

In [ ]:
y_valid.shape

In [ ]:
y_pred.shape

In [ ]:
col_1 = [int(round(p)) for p in y_pred[:,0]]


In [ ]:
col_2 = [int(round(p)) for p in y_pred[:,1]]

In [ ]:
col_1 = np.array(col_1 ,ndmin = 2)
col_2 = np.array(col_2,ndmin = 2)

In [ ]:
col_2.shape

In [ ]:
results =np.transpose(np.concatenate([col_1,col_2],axis=0))
results.shape

In [ ]:
print(classification_report(results,y_valid))

In [ ]:
confusion_matrix(results[:,0],y_valid[:,0])